# Twin4Build: CO2 Controller Example

This notebook demonstrates how to use the Twin4Build package to create and simulate a CO2 controller model for a building space. We'll go through the process step-by-step, explaining each part along the way.

## 1. Import Required Libraries

First, we import the necessary libraries and modules.

In [ ]:
%pip install git+https://github.com/JBjoernskov/Twin4Build.git
import twin4build as tb
import datetime
from dateutil import tz
import twin4build.utils.plot.plot as plot
import matplotlib.pyplot as plt
import os

## 2. Define Model Components and Connections

Now we define the components of our CO2 controller model and their connections.
For convenience, we can pack all these definitions into the "fcn" function, which will be used to create the model.

In [ ]:
def fcn(self):
    ##############################################################
    ################## First, define components ##################
    ##############################################################
    occupancy_schedule = tb.ScheduleSystem(
        weekDayRulesetDict={
            "ruleset_default_value": 0,
            "ruleset_start_minute": [0, 0, 0, 0, 0, 0, 0],
            "ruleset_end_minute": [0, 0, 0, 0, 0, 0, 0],
            "ruleset_start_hour": [6, 7, 8, 12, 14, 16, 18],
            "ruleset_end_hour": [7, 8, 12, 14, 16, 18, 22],
            "ruleset_value": [3, 5, 20, 25, 27, 7, 3]},
        add_noise=True,
        saveSimulationResult=True,
        id="Occupancy schedule")
    
    co2_setpoint_schedule = tb.ScheduleSystem(
        weekDayRulesetDict={
            "ruleset_default_value": 900,
            "ruleset_start_minute": [],
            "ruleset_end_minute": [],
            "ruleset_start_hour": [],
            "ruleset_end_hour": [],
            "ruleset_value": []},
        saveSimulationResult=True,
        id="CO2 setpoint schedule")

    co2_property = tb.Co2()
    co2_controller = tb.PIDControllerSystem(
        observes=co2_property,
        K_p=-0.001,
        K_i=-0.001,
        K_d=0,
        saveSimulationResult=True,
        id="CO2 controller")

    supply_damper = tb.DamperSystem(
        nominalAirFlowRate=tb.PropertyValue(hasValue=1.6),
        a=5,
        saveSimulationResult=True,
        id="Supply damper")

    return_damper = tb.DamperSystem(
        nominalAirFlowRate=tb.PropertyValue(hasValue=1.6),
        a=5,
        saveSimulationResult=True,
        id="Return damper")

    space = tb.BuildingSpaceCo2System(
        airVolume=466.54,
        outdoorCo2Concentration=500,
        infiltration=0.005,
        generationCo2Concentration=0.0042*1000*1.225,
        saveSimulationResult=True,
        id="Space")
    co2_property.isPropertyOf = space

    #################################################################
    ################## Add connections to the model #################
    #################################################################
    self.add_connection(co2_controller, supply_damper,
                         "inputSignal", "damperPosition")
    self.add_connection(co2_controller, return_damper,
                         "inputSignal", "damperPosition")
    self.add_connection(supply_damper, space,
                         "airFlowRate", "supplyAirFlowRate")
    self.add_connection(return_damper, space,
                         "airFlowRate", "returnAirFlowRate")
    self.add_connection(occupancy_schedule, space,
                         "scheduleValue", "numberOfPeople")
    self.add_connection(space, co2_controller,
                         "indoorCo2Concentration", "actualValue")
    self.add_connection(co2_setpoint_schedule, co2_controller,
                         "scheduleValue", "setpointValue")

## 3. Create and Load the Model

Now we create a model instance and load it with the components and connections we defined.

In [ ]:
model = tb.Model(id="co2_control")
model.load(fcn=fcn)
system_graph = os.path.join(model.graph_path, "system_graph.png")
image = plt.imread(system_graph)
plt.figure()
plt.imshow(image)
plt.axis('off')
plt.show()

## 4. Set Up Simulation Parameters and Run Simulation

We set up the simulation parameters and run the simulation for our CO2 controller model.

In [ ]:
simulator = tb.Simulator()
stepSize = 600 #Seconds
startTime = datetime.datetime(year=2024, month=1, day=10, hour=0, minute=0, second=0, tzinfo=tz.gettz("Europe/Copenhagen"))
endTime = datetime.datetime(year=2024, month=1, day=12, hour=0, minute=0, second=0, tzinfo=tz.gettz("Europe/Copenhagen"))

# Simulate the model
simulator.simulate(model,
                    stepSize=stepSize,
                    startTime = startTime,
                    endTime = endTime)

## 5. Visualize Results

We use the built-in plotting functions to visualize the simulation results.

In [ ]:
plot.plot_damper(model, simulator, "Supply damper", show=True)

In [ ]:
plot.plot_CO2_controller(model, simulator, "CO2 controller", show=True)

In [ ]:
plot.plot_space_CO2(model, simulator, "Space", show=True)

## 6. Interpreting the Results

The plots show:

1. Supply Damper Position: This graph illustrates how the damper position changes over time in response to CO2 levels.

2. CO2 Controller: This plot shows the actual CO2 concentration (blue line) compared to the setpoint (orange line), and the controller output (green line).

These visualizations help us understand how the CO2 controller is managing the indoor air quality based on occupancy and setpoints.

## 7. Next Steps

To further explore this model, you could:
- Adjust the PID controller parameters to see how they affect the system response
- Modify the occupancy schedule to simulate different usage patterns
- Change the CO2 setpoint to observe its impact on ventilation behavior

This example demonstrates the power of Twin4Build for modeling and simulating building control systems.